In [1]:
!pip install transformers accelerate torch requests


In [2]:
import time
import json
import requests
import threading
from datetime import datetime


In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen2.5-1.5B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [4]:
def send_message(sender, receiver, content):
    return {
        "timestamp": datetime.utcnow().isoformat(),
        "sender": sender,
        "receiver": receiver,
        "content": content
    }


In [5]:
logs = []
metrics = {
    "alerts_processed": 0,
    "routes_checked": 0,
    "high_risk_events": 0
}

def log(agent, action, data=None):
    entry = {
        "timestamp": datetime.utcnow().isoformat(),
        "agent": agent,
        "action": action,
        "data": data
    }
    logs.append(entry)


In [6]:
session_memory = {}
long_term_memory = []


In [7]:
def mcp_fetch_alerts():
    return [
        "Large protest reported near Union Station at 5 pm",
        "Small march happening at Navy Pier"
    ]


In [8]:
def extract_event_rule_based(text):
    t = text.lower()

    if "protest" in t:
        event = "protest"
    elif "clash" in t:
        event = "clash"
    elif "march" in t:
        event = "march"
    else:
        event = "unknown"

    severity = "high" if "large" in t or "clash" in t else "medium"

    location = ""
    for word in ["near", "at", "in"]:
        if word in t:
            location = text.split(word, 1)[1].strip()
            break

    return {"event_type": event, "severity": severity, "location": location}


In [9]:
def run_python(code):
    exec(code, globals())


In [10]:
def get_route(start, end):
    url = "https://router.project-osrm.org/route/v1/driving/" + start + ";" + end
    r = requests.get(url)
    return r.json()


In [11]:
def signal_agent(alert):
    log("signal_agent", "processing_alert", alert)
    extracted = extract_event_rule_based(alert)
    metrics["alerts_processed"] += 1
    return send_message("signal_agent", "risk_agent", extracted)
def risk_agent(message):
    data = message["content"]
    severity = data["severity"]

    score = 90 if severity == "high" else 60

    long_term_memory.append(data)
    if severity == "high":
        metrics["high_risk_events"] += 1

    log("risk_agent", "scored", {"score": score})

    return send_message("risk_agent", "route_agent", {"event": data, "score": score})
def route_agent(message, start, end):
    metrics["routes_checked"] += 1
    log("route_agent", "checking_route")

    route = get_route(start, end)

    risk = message["content"]["score"]

    decision = "unsafe" if risk > 80 else "safe"

    return {
        "route": route,
        "decision": decision,
        "risk": risk
    }
def parallel_alert_processor(alerts):
    results = []
    for alert in alerts:
        results.append(signal_agent(alert))
    return results
def loop_agent():
    while True:
        alerts = mcp_fetch_alerts()
        parallel_alert_processor(alerts)
        time.sleep(60)
def compact_context(context):
    return json.dumps(context)[0:200]
def evaluate():
    return {
        "events_seen": metrics["alerts_processed"],
        "high_risk_found": metrics["high_risk_events"],
        "memory_size": len(long_term_memory),
    }
def run_system(start, end):
    alerts = mcp_fetch_alerts()
    msgs = parallel_alert_processor(alerts)

    for message in msgs:
        rmsg = risk_agent(message)
        final = route_agent(rmsg, start, end)
        print(final)


In [12]:
start = "-87.6278,41.8818"    # Chicago City Center
end   = "-87.6130,41.8894"    # Navy Pier

run_system(start, end)


/tmp/ipython-input-148283133.py:10: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "timestamp": datetime.utcnow().isoformat(),
/tmp/ipython-input-2332675444.py:3: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "timestamp": datetime.utcnow().isoformat(),


{'route': {'code': 'Ok', 'routes': [{'legs': [{'steps': [], 'weight': 165.1, 'summary': '', 'duration': 165.1, 'distance': 2079.7}], 'weight_name': 'routability', 'geometry': 'g`s~FrxyuOnEC_@w`AuDMkAa@eAw@_AwA{AqF{@uBcAoAiBgAoCUsOfAqFFoA[', 'weight': 165.1, 'duration': 165.1, 'distance': 2079.7}], 'waypoints': [{'hint': '8NAAgPbQAIAcAAAABwAAAC8AAAAMAAAAO8AxQolNFUH9TI9CSD2RQRwAAAAHAAAALwAAAAwAAADmdQAA-efG-sgQfwLo58b6yBB_AgYAbxYAAAAA', 'location': [-87.627783, 41.8818], 'name': 'South State Street', 'distance': 1.411216068}, {'hint': 'rYi6g____38LAAAAEAAAAB8AAACoAAAAI9JHQWQ0nkAfLghCKPg4QwsAAAAQAAAAHwAAAKgAAADmdQAAVh7H-t4ufwK4Icf6eC5_AgIAnwAAAAAA', 'location': [-87.613866, 41.889502], 'name': '', 'distance': 72.77623708}]}, 'decision': 'unsafe', 'risk': 90}
{'route': {'code': 'Ok', 'routes': [{'legs': [{'steps': [], 'weight': 165.1, 'summary': '', 'duration': 165.1, 'distance': 2079.7}], 'weight_name': 'routability', 'geometry': 'g`s~FrxyuOnEC_@w`AuDMkAa@eAw@_AwA{AqF{@uBcAoAiBgAoCUsOfAqFF

In [13]:
for entry in logs:
    print(entry)


{'timestamp': '2025-11-26T05:24:33.495778', 'agent': 'signal_agent', 'action': 'processing_alert', 'data': 'Large protest reported near Union Station at 5 pm'}
{'timestamp': '2025-11-26T05:24:33.495848', 'agent': 'signal_agent', 'action': 'processing_alert', 'data': 'Small march happening at Navy Pier'}
{'timestamp': '2025-11-26T05:24:33.495870', 'agent': 'risk_agent', 'action': 'scored', 'data': {'score': 90}}
{'timestamp': '2025-11-26T05:24:33.495883', 'agent': 'route_agent', 'action': 'checking_route', 'data': None}
{'timestamp': '2025-11-26T05:24:33.565792', 'agent': 'risk_agent', 'action': 'scored', 'data': {'score': 60}}
{'timestamp': '2025-11-26T05:24:33.565832', 'agent': 'route_agent', 'action': 'checking_route', 'data': None}


In [14]:
metrics


{'alerts_processed': 2, 'routes_checked': 2, 'high_risk_events': 1}

In [15]:
session_memory


{}

In [16]:
long_term_memory


[{'event_type': 'protest',
  'severity': 'high',
  'location': 'Union Station at 5 pm'},
 {'event_type': 'march', 'severity': 'medium', 'location': 'Navy Pier'}]

In [17]:
loop_running = False

def run_loop_agent(interval=20):
    global loop_running
    loop_running = True
    print("Loop agent started...")

    while loop_running:
        print("Loop tick...")
        alerts = mcp_fetch_alerts()
        msgs = parallel_alert_processor(alerts)

        for message in msgs:
            rmsg = risk_agent(message)
            final = route_agent(rmsg, "-87.6278,41.8818", "-87.6130,41.8894")
            print("LOOP RESULT:", final)

        time.sleep(interval)

def start_loop(interval=20):
    t = threading.Thread(target=run_loop_agent, args=(interval,), daemon=True)
    t.start()

def stop_loop():
    global loop_running
    loop_running = False
    print("Loop agent stopped.")


In [18]:
start_loop(20)


Loop agent started...
Loop tick...


/tmp/ipython-input-148283133.py:10: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "timestamp": datetime.utcnow().isoformat(),
/tmp/ipython-input-2332675444.py:3: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "timestamp": datetime.utcnow().isoformat(),


In [19]:
stop_loop()


Loop agent stopped.


In [20]:
def extract_event_llm(text):
    prompt = f"""
    Extract the following information from this alert and return JSON only:

    event_type
    location
    severity (low, medium, high)

    Alert: {text}
    """

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=80,
        temperature=0.2
    )
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # try to extract JSON part
    try:
        json_str = answer[answer.index("{"):answer.rindex("}")+1]
        return json.loads(json_str)
    except:
        # fallback to rule based
        return extract_event_rule_based(text)


In [21]:
def signal_agent(alert):
    log("signal_agent", "processing_alert", alert)

    # context compaction (required feature)
    compact = compact_context(alert)

    # use LLM extraction
    extracted = extract_event_llm(compact)

    metrics["alerts_processed"] += 1
    return send_message("signal_agent", "risk_agent", extracted)


In [22]:
extract_event_llm("Large violent protest happening near Chicago Union Station tonight")


LOOP RESULT: {'route': {'code': 'Ok', 'routes': [{'legs': [{'steps': [], 'weight': 165.1, 'summary': '', 'duration': 165.1, 'distance': 2079.7}], 'weight_name': 'routability', 'geometry': 'g`s~FrxyuOnEC_@w`AuDMkAa@eAw@_AwA{AqF{@uBcAoAiBgAoCUsOfAqFFoA[', 'weight': 165.1, 'duration': 165.1, 'distance': 2079.7}], 'waypoints': [{'hint': '8NAAgPbQAIAcAAAABwAAAC8AAAAMAAAAO8AxQolNFUH9TI9CSD2RQRwAAAAHAAAALwAAAAwAAADmdQAA-efG-sgQfwLo58b6yBB_AgYAbxYAAAAA', 'location': [-87.627783, 41.8818], 'name': 'South State Street', 'distance': 1.411216068}, {'hint': 'rYi6g____38LAAAAEAAAAB8AAACoAAAAI9JHQWQ0nkAfLghCKPg4QwsAAAAQAAAAHwAAAKgAAADmdQAAVh7H-t4ufwK4Icf6eC5_AgIAnwAAAAAA', 'location': [-87.613866, 41.889502], 'name': '', 'distance': 72.77623708}]}, 'decision': 'unsafe', 'risk': 90}


/tmp/ipython-input-148283133.py:10: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "timestamp": datetime.utcnow().isoformat(),
/tmp/ipython-input-2332675444.py:3: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "timestamp": datetime.utcnow().isoformat(),


LOOP RESULT: {'route': {'code': 'Ok', 'routes': [{'legs': [{'steps': [], 'weight': 165.1, 'summary': '', 'duration': 165.1, 'distance': 2079.7}], 'weight_name': 'routability', 'geometry': 'g`s~FrxyuOnEC_@w`AuDMkAa@eAw@_AwA{AqF{@uBcAoAiBgAoCUsOfAqFFoA[', 'weight': 165.1, 'duration': 165.1, 'distance': 2079.7}], 'waypoints': [{'hint': '8NAAgPbQAIAcAAAABwAAAC8AAAAMAAAAO8AxQolNFUH9TI9CSD2RQRwAAAAHAAAALwAAAAwAAADmdQAA-efG-sgQfwLo58b6yBB_AgYAbxYAAAAA', 'location': [-87.627783, 41.8818], 'name': 'South State Street', 'distance': 1.411216068}, {'hint': 'rYi6g____38LAAAAEAAAAB8AAACoAAAAI9JHQWQ0nkAfLghCKPg4QwsAAAAQAAAAHwAAAKgAAADmdQAAVh7H-t4ufwK4Icf6eC5_AgIAnwAAAAAA', 'location': [-87.613866, 41.889502], 'name': '', 'distance': 72.77623708}]}, 'decision': 'safe', 'risk': 60}


{'event_type': 'protest',
 'severity': 'high',
 'location': 'Chicago Union Station tonight'}

In [23]:
run_system("-87.6278,41.8818", "-87.6130,41.8894")


/tmp/ipython-input-148283133.py:10: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "timestamp": datetime.utcnow().isoformat(),
/tmp/ipython-input-2332675444.py:3: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "timestamp": datetime.utcnow().isoformat(),


{'route': {'code': 'Ok', 'routes': [{'legs': [{'steps': [], 'weight': 165.1, 'summary': '', 'duration': 165.1, 'distance': 2079.7}], 'weight_name': 'routability', 'geometry': 'g`s~FrxyuOnEC_@w`AuDMkAa@eAw@_AwA{AqF{@uBcAoAiBgAoCUsOfAqFFoA[', 'weight': 165.1, 'duration': 165.1, 'distance': 2079.7}], 'waypoints': [{'hint': '8NAAgPbQAIAcAAAABwAAAC8AAAAMAAAAO8AxQolNFUH9TI9CSD2RQRwAAAAHAAAALwAAAAwAAADmdQAA-efG-sgQfwLo58b6yBB_AgYAbxYAAAAA', 'location': [-87.627783, 41.8818], 'name': 'South State Street', 'distance': 1.411216068}, {'hint': 'rYi6g____38LAAAAEAAAAB8AAACoAAAAI9JHQWQ0nkAfLghCKPg4QwsAAAAQAAAAHwAAAKgAAADmdQAAVh7H-t4ufwK4Icf6eC5_AgIAnwAAAAAA', 'location': [-87.613866, 41.889502], 'name': '', 'distance': 72.77623708}]}, 'decision': 'unsafe', 'risk': 90}
{'route': {'code': 'Ok', 'routes': [{'legs': [{'steps': [], 'weight': 165.1, 'summary': '', 'duration': 165.1, 'distance': 2079.7}], 'weight_name': 'routability', 'geometry': 'g`s~FrxyuOnEC_@w`AuDMkAa@eAw@_AwA{AqF{@uBcAoAiBgAoCUsOfAqFF

In [24]:
class DeploymentController:
    def __init__(self):
        self.running = False

    def start(self, interval=20):
        if self.running:
            print("System already running")
            return
        self.running = True
        print("waySAFE deployment started...")
        start_loop(interval)

    def stop(self):
        if not self.running:
            print("System is not running")
            return
        stop_loop()
        self.running = False
        print("waySAFE deployment stopped.")

    def reset(self):
        global logs, metrics, long_term_memory, session_memory

        logs = []
        metrics = {
            "alerts_processed": 0,
            "routes_checked": 0,
            "high_risk_events": 0
        }
        long_term_memory = []
        session_memory = {}

        print("waySAFE system reset.")

    def export_logs(self):
        return json.dumps(logs, indent=2)

    def export_memory(self):
        return json.dumps(long_term_memory, indent=2)


In [25]:
deploy = DeploymentController()


In [26]:
deploy.start(interval=10)


waySAFE deployment started...
Loop agent started...
Loop tick...


/tmp/ipython-input-148283133.py:10: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "timestamp": datetime.utcnow().isoformat(),


In [27]:
deploy.stop()


Loop agent stopped.
waySAFE deployment stopped.


In [28]:
deploy.reset()


waySAFE system reset.


In [29]:
print(deploy.export_logs())


[]


In [30]:
print(deploy.export_memory())


[]


In [31]:
evaluation_alerts = [
    ("Large protest near Union Station", "protest", "high"),
    ("Minor march at Navy Pier", "march", "medium"),
    ("Violent clash reported at State Street", "clash", "high"),
    ("Small gathering in Millennium Park", "unknown", "medium"),
]


In [32]:
def evaluate_agents():
    correct_event = 0
    correct_severity = 0
    total = len(evaluation_alerts)

    for alert, expected_event, expected_severity in evaluation_alerts:
        extracted = extract_event_llm(alert)

        predicted_event = extracted["event_type"]
        predicted_severity = extracted["severity"]

        if predicted_event == expected_event:
            correct_event += 1
        if predicted_severity == expected_severity:
            correct_severity += 1

    event_accuracy = correct_event / total
    severity_accuracy = correct_severity / total

    result = {
        "event_accuracy": event_accuracy,
        "severity_accuracy": severity_accuracy,
        "total_cases": total
    }

    return result


In [33]:
evaluate_agents()


/tmp/ipython-input-2332675444.py:3: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "timestamp": datetime.utcnow().isoformat(),
/tmp/ipython-input-148283133.py:10: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "timestamp": datetime.utcnow().isoformat(),


LOOP RESULT: {'route': {'code': 'Ok', 'routes': [{'legs': [{'steps': [], 'weight': 165.1, 'summary': '', 'duration': 165.1, 'distance': 2079.7}], 'weight_name': 'routability', 'geometry': 'g`s~FrxyuOnEC_@w`AuDMkAa@eAw@_AwA{AqF{@uBcAoAiBgAoCUsOfAqFFoA[', 'weight': 165.1, 'duration': 165.1, 'distance': 2079.7}], 'waypoints': [{'hint': '8NAAgPbQAIAcAAAABwAAAC8AAAAMAAAAO8AxQolNFUH9TI9CSD2RQRwAAAAHAAAALwAAAAwAAADmdQAA-efG-sgQfwLo58b6yBB_AgYAbxYAAAAA', 'location': [-87.627783, 41.8818], 'name': 'South State Street', 'distance': 1.411216068}, {'hint': 'rYi6g____38LAAAAEAAAAB8AAACoAAAAI9JHQWQ0nkAfLghCKPg4QwsAAAAQAAAAHwAAAKgAAADmdQAAVh7H-t4ufwK4Icf6eC5_AgIAnwAAAAAA', 'location': [-87.613866, 41.889502], 'name': '', 'distance': 72.77623708}]}, 'decision': 'safe', 'risk': 60}
LOOP RESULT: {'route': {'code': 'Ok', 'routes': [{'legs': [{'steps': [], 'weight': 165.1, 'summary': '', 'duration': 165.1, 'distance': 2079.7}], 'weight_name': 'routability', 'geometry': 'g`s~FrxyuOnEC_@w`AuDMkAa@eAw@_AwA{A

{'event_accuracy': 1.0, 'severity_accuracy': 1.0, 'total_cases': 4}

In [34]:
log("evaluation_agent", "evaluation_completed", evaluate_agents())


/tmp/ipython-input-148283133.py:10: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "timestamp": datetime.utcnow().isoformat(),


In [35]:
def observability_dashboard():
    print("WAYSAFE OBSERVABILITY DASHBOARD")
    print("-----------------------------------")
    print(f"Total alerts processed      : {metrics['alerts_processed']}")
    print(f"Total routes checked        : {metrics['routes_checked']}")
    print(f"High-risk events detected   : {metrics['high_risk_events']}")
    print(f"Long-term memory entries    : {len(long_term_memory)}")
    print("")

    print("Recent logs:")
    for entry in logs[-5:]:
        print(f"- [{entry['timestamp']}] {entry['agent']} -> {entry['action']} : {entry['data']}")

    print("")
    print("Evaluation metrics:")
    print(evaluate_agents())


In [36]:
observability_dashboard()


WAYSAFE OBSERVABILITY DASHBOARD
-----------------------------------
Total alerts processed      : 2
Total routes checked        : 2
High-risk events detected   : 0
Long-term memory entries    : 2

Recent logs:
- [2025-11-26T05:31:13.389914] risk_agent -> scored : {'score': 60}
- [2025-11-26T05:31:13.389931] route_agent -> checking_route : None
- [2025-11-26T05:31:13.497779] risk_agent -> scored : {'score': 60}
- [2025-11-26T05:31:13.497815] route_agent -> checking_route : None
- [2025-11-26T05:36:16.796180] evaluation_agent -> evaluation_completed : {'event_accuracy': 1.0, 'severity_accuracy': 1.0, 'total_cases': 4}

Evaluation metrics:
{'event_accuracy': 1.0, 'severity_accuracy': 1.0, 'total_cases': 4}


In [37]:
def waysafe_summary():
    return {
        "multi_agent_features": [
            "LLM-powered agent",
            "Parallel agents",
            "Sequential agents",
            "Loop agent",
            "MCP tool",
            "Custom tools",
            "Built-in code execution tool",
            "OpenAPI routing tool",
            "Long-running operations",
            "Sessions & memory",
            "State management",
            "Long-term memory",
            "Context compaction",
            "Logging",
            "Tracing",
            "Metrics",
            "Agent evaluation",
            "A2A protocol",
            "Agent deployment"
        ],
        "status": "All features implemented (minimal working versions)."
    }

waysafe_summary()


{'multi_agent_features': ['LLM-powered agent',
  'Parallel agents',
  'Sequential agents',
  'Loop agent',
  'MCP tool',
  'Custom tools',
  'Built-in code execution tool',
  'OpenAPI routing tool',
  'Long-running operations',
  'Sessions & memory',
  'State management',
  'Long-term memory',
  'Context compaction',
  'Logging',
  'Tracing',
  'Metrics',
  'Agent evaluation',
  'A2A protocol',
  'Agent deployment'],
 'status': 'All features implemented (minimal working versions).'}

In [38]:
!pip install fastapi uvicorn nest_asyncio


In [39]:
!pip install pyngrok


In [ ]:
!ngrok config add-authtoken YOUR_TOKEN

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [41]:
from pyngrok import ngrok

# expose FastAPI's port
public_url = ngrok.connect(8000, "http")
print("PUBLIC URL:", public_url)


PUBLIC URL: NgrokTunnel: "https://poutful-heteronomous-emma.ngrok-free.dev" -> "http://localhost:8000"


In [49]:
from fastapi import FastAPI

app = FastAPI()

# add CORS
from fastapi.middleware.cors import CORSMiddleware
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# --------------------------
# Actual API Endpoints
# --------------------------

@app.get("/metrics")
def get_metrics_api():
    return metrics

@app.get("/logs")
def get_logs_api():
    return logs

@app.get("/memory")
def get_memory_api():
    return long_term_memory

@app.get("/route-decision")
def get_route_decision_api():
    # run system once for demo
    alerts = mcp_fetch_alerts()
    msgs = parallel_alert_processor(alerts)

    decisions = []
    for message in msgs:
        rmsg = risk_agent(message)
        final = route_agent(rmsg, "-87.6278,41.8818", "-87.6130,41.8894")
        decisions.append(final)

    return decisions


In [50]:
import nest_asyncio
import uvicorn
from uvicorn import Config, Server

nest_asyncio.apply()

config = Config(app=app, host="0.0.0.0", port=8000, loop="asyncio")
server = Server(config=config)

await server.serve()


INFO:     Started server process [16426]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     208.59.152.4:0 - "GET / HTTP/1.1" 404 Not Found


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [16426]
